In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import MinMaxScaler

2024-10-04 18:12:45.444277: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-04 18:12:45.480924: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-04 18:12:46.026079: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Reading the input data and targets data Simulated dataset 
# Example data: LBA, Size, r/w, Sequential
data = np.loadtxt('input_data.txt')
targets = np.loadtxt('target_data.txt')
targets = targets.reshape(-1, 1)
full_data = np.concatenate((data, targets), axis=1)

In [3]:
# lba = data[:, 0]  # Logical Block Address
# size = data[:, 1]  # Size of the I/O request
# rw = data[:, 2]  # Read or Write
# is_sequential = data[:, 3]  # Is Sequential (1 or 0)

In [4]:
# features = np.column_stack((lba, size, rw, is_sequential))
# # Apply Min-Max Scaling
# scaler = MinMaxScaler()
# normalized_features = scaler.fit_transform(features)
scaler = MinMaxScaler()
normalized_features = scaler.fit_transform(full_data)
normalized_data = normalized_features[:, :-1]  # All columns except the last one
normalized_targets = normalized_features[:, -1]  # Last column
print("Normalized Features:")
print(normalized_features)

Normalized Features:
[[0.00000000e+00 2.34558249e-03 9.87741832e-03 0.00000000e+00
  0.00000000e+00]
 [2.98023233e-08 2.34558249e-03 1.48280312e-02 1.00000000e+00
  0.00000000e+00]
 [8.94069698e-08 2.34558249e-03 1.97786441e-02 0.00000000e+00
  0.00000000e+00]
 ...
 [5.00976577e-01 2.34558249e-03 6.54780440e-03 0.00000000e+00
  6.37653842e-07]
 [0.00000000e+00 2.34558249e-03 6.54779450e-03 0.00000000e+00
  9.96817470e-01]
 [0.00000000e+00 9.92963253e-02 6.54780440e-03 0.00000000e+00
  6.37653842e-07]]


In [5]:
# Prepare the data for GRU
X = np.expand_dims(normalized_data, axis=0)  # Adding batch dimension
y = np.expand_dims(normalized_targets, axis=0)  # Adding batch dimension

In [6]:
# Define the GRU model in TensorFlow
model = models.Sequential()
model.add(layers.GRU(100, return_sequences=True, input_shape=(None, X.shape[2])))
model.add(layers.GRU(50, return_sequences=False))
model.add(layers.Dense(1))

2024-10-04 18:12:56.858806: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22270 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:17:00.0, compute capability: 8.6
2024-10-04 18:12:56.859323: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22061 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


In [7]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mse')

In [8]:
# Train the model (for simplicity, using a few epochs)
model.fit(X, y, epochs=250)

Epoch 1/250


2024-10-04 18:13:09.104858: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-10-04 18:13:28.456596: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-10-04 18:13:28.499542: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fd6d80d8010 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-10-04 18:13:28.499561: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-10-04 18:13:28.499565: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-10-04 18:13:28.503179: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-04 

1/1 [==============================] - 45s 45s/step - loss: 0.0170
Epoch 2/250
1/1 [==============================] - 42s 42s/step - loss: 6.7303
Epoch 3/250
1/1 [==============================] - 42s 42s/step - loss: 0.2001
Epoch 4/250


In [ ]:
# Model summary
model.summary()


In [ ]:
# Get the GRU layer weights
gru_layer = model.layers[0]
weights = gru_layer.get_weights()

In [ ]:
# Save weights to use them in the C code
np.savez('gru_weights.npz', Wz=weights[0], Uz=weights[1], bz=weights[2])
print("GRU weights saved to 'gru_weights.npz'")

In [ ]:
# Predict on the data (inference)
predictions = model.predict(X)
print("Predictions:", predictions)